In [1]:
import numpy as np
import gc
import pandas as pd
import xarray as xr


In [2]:

# Load the parquet files
atlantic_31_pred = pd.read_parquet('atlantic_31_pred.parquet')
atlantic_42_pred = pd.read_parquet('atlantic_42_pred.parquet')

# Print basic information about the loaded data
print("Atlantic 31 shape:", atlantic_31_pred.shape)
print("Atlantic 42 shape:", atlantic_42_pred.shape)


Atlantic 31 shape: (359424000, 11)
Atlantic 42 shape: (359424000, 11)


In [3]:
# import h5py
# import numpy as np
# import pandas as pd

# def read_mat_v73(mat_path: str) -> dict:
#     """读取MATLAB v7.3文件 (适配NumPy 1.20+版本)"""
#     data = {}
#     with h5py.File(mat_path, 'r') as f:
#         def visit_handler(name, obj):
#             if isinstance(obj, h5py.Dataset):
#                 # 处理对象类型数据集
#                 arr = np.array(obj)
#                 # 类型检查改用安全方式
#                 if arr.dtype == np.dtype('O') or arr.dtype.type is np.object_:
#                     arr = arr[0]  # 解包MATLAB容器
#                 data[name.split('/')[-1]] = arr
#         f.visititems(visit_handler)
#     return data

# data= read_mat_v73("atlantic_42.mat")


In [4]:
# original_shape = (20, 104, 120, 1440)
# dura_mhw = atlantic_42_pred[['dura_mhw']]
# # dura_mhw_trans = (
# #     dura_mhw
# #     .reshape(original_shape[::-1], order="F")  # 反转形状为 (D, C, B, A)
# #     .transpose(3, 2, 1, 0)                     # 转置轴为 (A, B, C, D), 此时存储顺序变为C
# #     .copy(order='C')                           # 确保内存连续性（可选）
# # )
# dura_mhw_trans=dura_mhw.to_numpy().reshape(original_shape, order='F')
# dura_mhw_trans

In [5]:
# atlantic_31_pred

In [6]:
# Split the data into the different prediction types
atlantic_31_pred_mhw_flat = atlantic_31_pred[['pred_mhw']]
atlantic_31_pred_clm_flat = atlantic_31_pred[['pred_clim']]
atlantic_31_pred_nan_flat = atlantic_31_pred[['pred_no_mhw']]
atlantic_42_pred_mhw_flat = atlantic_42_pred[['pred_mhw']]
atlantic_42_pred_clm_flat = atlantic_42_pred[['pred_clim']]
atlantic_42_pred_nan_flat = atlantic_42_pred[['pred_no_mhw']]
# del atlantic_31_pred, atlantic_42_pred
# gc.collect()

In [7]:
# to_numpy().reshape the data into the original shape
original_shape=(20, 104, 120, 1440)
atlantic_31_pred_mhw_np=atlantic_31_pred_mhw_flat.to_numpy().reshape(original_shape, order='F')
atlantic_31_pred_clm_np=atlantic_31_pred_clm_flat.to_numpy().reshape(original_shape, order='F')
atlantic_31_pred_nan_np=atlantic_31_pred_nan_flat.to_numpy().reshape(original_shape, order='F')
atlantic_42_pred_mhw_np=atlantic_42_pred_mhw_flat.to_numpy().reshape(original_shape, order='F')
atlantic_42_pred_clm_np=atlantic_42_pred_clm_flat.to_numpy().reshape(original_shape, order='F')
atlantic_42_pred_nan_np=atlantic_42_pred_nan_flat.to_numpy().reshape(original_shape, order='F')
# atlantic_31_pred_mhw_np=
# del atlantic_31_pred_mhw_flat, atlantic_31_pred_clm_flat, atlantic_31_pred_nan_flat, atlantic_42_pred_mhw_flat, atlantic_42_pred_clm_flat, atlantic_42_pred_nan_flat
# gc.collect()



In [8]:
# Create xarray DataArrays
Latitudes = np.linspace(60, 90, 120)
Longitudes = np.linspace(-180, 180, 1440)
Years = np.linspace(1982, 2085, 104)
times = np.linspace(0, 19, 20)

atlantic_31_pred_mhw_xr = xr.DataArray(atlantic_31_pred_mhw_np,dims=["time","Years","Latitude", "Longitude"],coords={"time": times, "Years": Years, "Latitude": Latitudes, "Longitude": Longitudes}, name="Mortality")
atlantic_31_pred_clm_xr = xr.DataArray(atlantic_31_pred_clm_np,dims=["time","Years","Latitude", "Longitude"],coords={"time": times, "Years": Years, "Latitude": Latitudes, "Longitude": Longitudes}, name="Mortality")
atlantic_31_pred_nan_xr = xr.DataArray(atlantic_31_pred_nan_np,dims=["time","Years","Latitude", "Longitude"],coords={"time": times, "Years": Years, "Latitude": Latitudes, "Longitude": Longitudes}, name="Mortality")
atlantic_42_pred_mhw_xr = xr.DataArray(atlantic_42_pred_mhw_np,dims=["time","Years","Latitude", "Longitude"],coords={"time": times, "Years": Years, "Latitude": Latitudes, "Longitude": Longitudes}, name="Mortality")
atlantic_42_pred_clm_xr = xr.DataArray(atlantic_42_pred_clm_np,dims=["time","Years","Latitude", "Longitude"],coords={"time": times, "Years": Years, "Latitude": Latitudes, "Longitude": Longitudes}, name="Mortality")
atlantic_42_pred_nan_xr = xr.DataArray(atlantic_42_pred_nan_np,dims=["time","Years","Latitude", "Longitude"],coords={"time": times, "Years": Years, "Latitude": Latitudes, "Longitude": Longitudes}, name="Mortality")
# del atlantic_31_pred_mhw_np, atlantic_31_pred_clm_np, atlantic_31_pred_nan_np, atlantic_42_pred_mhw_np, atlantic_42_pred_clm_np, atlantic_42_pred_nan_np
# gc.collect()

In [9]:
# Create xarray Datasets and save them to netcdf
atlantic_31_pred=xr.Dataset({"mhw":atlantic_31_pred_mhw_xr, "clm":atlantic_31_pred_clm_xr, "nan":atlantic_31_pred_nan_xr})
atlantic_42_pred=xr.Dataset({"mhw":atlantic_42_pred_mhw_xr, "clm":atlantic_42_pred_clm_xr, "nan":atlantic_42_pred_nan_xr})
# del atlantic_31_pred_mhw_xr, atlantic_31_pred_clm_xr, atlantic_31_pred_nan_xr, atlantic_42_pred_mhw_xr, atlantic_42_pred_clm_xr, atlantic_42_pred_nan_xr
# gc.collect()
atlantic_31_pred.to_netcdf("atlantic_31_pred.nc")
atlantic_42_pred.to_netcdf("atlantic_42_pred.nc")
print(atlantic_31_pred)

# all_vars = list(globals().keys())
# for var in all_vars:
#     if not var.startswith("__"):  
#         del globals()[var]


<xarray.Dataset> Size: 9GB
Dimensions:    (time: 20, Years: 104, Latitude: 120, Longitude: 1440)
Coordinates:
  * time       (time) float64 160B 0.0 1.0 2.0 3.0 4.0 ... 16.0 17.0 18.0 19.0
  * Years      (Years) float64 832B 1.982e+03 1.983e+03 ... 2.084e+03 2.085e+03
  * Latitude   (Latitude) float64 960B 60.0 60.25 60.5 60.76 ... 89.5 89.75 90.0
  * Longitude  (Longitude) float64 12kB -180.0 -179.7 -179.5 ... 179.7 180.0
Data variables:
    mhw        (time, Years, Latitude, Longitude) float64 3GB nan nan ... nan
    clm        (time, Years, Latitude, Longitude) float64 3GB nan nan ... nan
    nan        (time, Years, Latitude, Longitude) float64 3GB nan nan ... nan


In [10]:
import numpy as np
import gc
import pandas as pd
import xarray as xr

# Load the parquet files
polar_31_pred = pd.read_parquet('polar_31_pred.parquet')
polar_42_pred = pd.read_parquet('polar_42_pred.parquet')

# Print basic information about the loaded data
print("Polar 31 shape:", polar_31_pred.shape)
print("Polar 42 shape:", polar_42_pred.shape)

# Split the data into the different prediction types
polar_31_pred_mhw_flat = polar_31_pred[['pred_mhw']]
polar_31_pred_clm_flat = polar_31_pred[['pred_clim']]
polar_31_pred_nan_flat = polar_31_pred[['pred_no_mhw']]
polar_42_pred_mhw_flat = polar_42_pred[['pred_mhw']]
polar_42_pred_clm_flat = polar_42_pred[['pred_clim']]
polar_42_pred_nan_flat = polar_42_pred[['pred_no_mhw']]
del polar_31_pred, polar_42_pred
gc.collect()

# to_numpy().reshape the data into the original shape
original_shape=(20, 103, 120, 1440)
polar_31_pred_mhw_np=polar_31_pred_mhw_flat.to_numpy().reshape(original_shape, order='F')
polar_31_pred_clm_np=polar_31_pred_clm_flat.to_numpy().reshape(original_shape, order='F')
polar_31_pred_nan_np=polar_31_pred_nan_flat.to_numpy().reshape(original_shape, order='F')
polar_42_pred_mhw_np=polar_42_pred_mhw_flat.to_numpy().reshape(original_shape, order='F')
polar_42_pred_clm_np=polar_42_pred_clm_flat.to_numpy().reshape(original_shape, order='F')
polar_42_pred_nan_np=polar_42_pred_nan_flat.to_numpy().reshape(original_shape, order='F')
del polar_31_pred_mhw_flat, polar_31_pred_clm_flat, polar_31_pred_nan_flat, polar_42_pred_mhw_flat, polar_42_pred_clm_flat, polar_42_pred_nan_flat
gc.collect()

# Create xarray DataArrays
Latitudes = np.linspace(60, 90, 120)
Longitudes = np.linspace(-180, 180, 1440)
Years = np.linspace(1982, 2084, 103)
times = np.linspace(0, 19, 20)
polar_31_pred_mhw_xr = xr.DataArray(polar_31_pred_mhw_np,dims=["time","Years","Latitude", "Longitude"],coords={"time": times, "Years": Years, "Latitude": Latitudes, "Longitude": Longitudes}, name="Mortality")
polar_31_pred_clm_xr = xr.DataArray(polar_31_pred_clm_np,dims=["time","Years","Latitude", "Longitude"],coords={"time": times, "Years": Years, "Latitude": Latitudes, "Longitude": Longitudes}, name="Mortality")
polar_31_pred_nan_xr = xr.DataArray(polar_31_pred_nan_np,dims=["time","Years","Latitude", "Longitude"],coords={"time": times, "Years": Years, "Latitude": Latitudes, "Longitude": Longitudes}, name="Mortality")
polar_42_pred_mhw_xr = xr.DataArray(polar_42_pred_mhw_np,dims=["time","Years","Latitude", "Longitude"],coords={"time": times, "Years": Years, "Latitude": Latitudes, "Longitude": Longitudes}, name="Mortality")
polar_42_pred_clm_xr = xr.DataArray(polar_42_pred_clm_np,dims=["time","Years","Latitude", "Longitude"],coords={"time": times, "Years": Years, "Latitude": Latitudes, "Longitude": Longitudes}, name="Mortality")
polar_42_pred_nan_xr = xr.DataArray(polar_42_pred_nan_np,dims=["time","Years","Latitude", "Longitude"],coords={"time": times, "Years": Years, "Latitude": Latitudes, "Longitude": Longitudes}, name="Mortality")
del polar_31_pred_mhw_np, polar_31_pred_clm_np, polar_31_pred_nan_np, polar_42_pred_mhw_np, polar_42_pred_clm_np, polar_42_pred_nan_np
gc.collect()

# Create xarray Datasets and save them to netcdf
polar_31_pred=xr.Dataset({"mhw":polar_31_pred_mhw_xr, "clm":polar_31_pred_clm_xr, "nan":polar_31_pred_nan_xr})
polar_42_pred=xr.Dataset({"mhw":polar_42_pred_mhw_xr, "clm":polar_42_pred_clm_xr, "nan":polar_42_pred_nan_xr})
polar_31_pred.to_netcdf("polar_31_pred.nc")
polar_42_pred.to_netcdf("polar_42_pred.nc")
print(polar_31_pred)

all_vars = list(globals().keys())
for var in all_vars:
    if not var.startswith("__"):  
        del globals()[var]


Polar 31 shape: (355968000, 11)
Polar 42 shape: (355968000, 11)
<xarray.Dataset> Size: 9GB
Dimensions:    (time: 20, Years: 103, Latitude: 120, Longitude: 1440)
Coordinates:
  * time       (time) float64 160B 0.0 1.0 2.0 3.0 4.0 ... 16.0 17.0 18.0 19.0
  * Years      (Years) float64 824B 1.982e+03 1.983e+03 ... 2.083e+03 2.084e+03
  * Latitude   (Latitude) float64 960B 60.0 60.25 60.5 60.76 ... 89.5 89.75 90.0
  * Longitude  (Longitude) float64 12kB -180.0 -179.7 -179.5 ... 179.7 180.0
Data variables:
    mhw        (time, Years, Latitude, Longitude) float64 3GB nan nan ... nan
    clm        (time, Years, Latitude, Longitude) float64 3GB nan nan ... nan
    nan        (time, Years, Latitude, Longitude) float64 3GB nan nan ... nan
